#setup:
- connect to google drive in this step

In [3]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow
import random

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [6]:
def get_data(individual, bodypart, h5_file):
  mouse_data = h5_file.xs(individual,level='individuals',axis=1)
  out_data = mouse_data.xs(bodypart,level='bodyparts',axis=1)
  out_data.columns = out_data.columns.droplevel("scorer")
  output = out_data.copy()
  return output

In [7]:
def within_area(area_vector, input_coor):
  area_startx = area_vector[0]
  area_starty = area_vector[1]
  area_distx = area_vector[2]
  area_disty = area_vector[3]
  x = input_coor["x"].iloc[0]
  y = input_coor["y"].iloc[0]
  if (area_startx <= x <= (area_startx+area_distx)) and (area_starty <= y <= (area_starty+area_disty)):
    result = 1
  else:
    result = 0
  return result

In [8]:
def euclid_dist(point1_coor, point2_coor):
  point1 = np.array((point1_coor["x"].iloc[0], point1_coor["y"].iloc[0]))
  point2 = np.array((point2_coor["x"].iloc[0], point2_coor["y"].iloc[0]))
  output_dist = np.linalg.norm(point1 - point2)
  return output_dist

In [9]:
def euclid_angle(pointa_coor, pointb_coor, pointc_coor):
  #angle_{pointa, pointb, pointc}
  a = np.array((pointa_coor["x"].iloc[0], pointa_coor["y"].iloc[0]))
  b = np.array((pointb_coor["x"].iloc[0], pointb_coor["y"].iloc[0]))
  c = np.array((pointc_coor["x"].iloc[0], pointc_coor["y"].iloc[0]))

  ba = a - b
  bc = c - b

  ac_dot = np.dot(ba, bc)

  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
  angle = np.arccos(cosine_angle)

  result = np.degrees(angle)
  return result

#Open some dlc data:

In [10]:
video_name = 'PZ70_1'

# area_vec = [area_startx, area_starty, area_distx, area_disty]
female_side_mat = scipy.io.loadmat(project_path + "/temp_classifier_testing/" + video_name + "_female_side.mat")
female_side_vec = female_side_mat['croprect'][0]
male_side_mat = scipy.io.loadmat(project_path + "/temp_classifier_testing/" + video_name + "_male_side.mat")
male_side_vec = male_side_mat['croprect'][0]

h5_file = pd.read_hdf(project_path +'/temp_classifier_testing/'+ video_name + 'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered.h5')
[nframes, ncols] = h5_file.shape

#open classifer model

In [11]:
#Importing essential libraries
import matplotlib.pyplot as plt
from statistics import mean
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [12]:
# load the model
classifier_file = open(project_path + '/classifiers/' +'classifer_temporal2.pickle', 'rb')
classifier = pickle.load(classifier_file)
classifier_file.close()

#analysis setup

In [13]:
# get all data ready for accessing as needed later on
mouse1_feature_points = {}
mouse1_feature_points['snout'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['shoulder'] = get_data('mus1', 'shoulder', h5_file)
mouse1_feature_points['spine1'] = get_data('mus1', 'spine1', h5_file)
mouse1_feature_points['spine2'] = get_data('mus1', 'spine2', h5_file)
mouse1_feature_points['spine3'] = get_data('mus1', 'spine3', h5_file)
mouse1_feature_points['spine4'] = get_data('mus1', 'spine4', h5_file)
mouse1_feature_points['tailbase'] = get_data('mus1', 'tailbase', h5_file)

mouse2_feature_points = {}
mouse2_feature_points['snout'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['shoulder'] = get_data('mus2', 'shoulder', h5_file)
mouse2_feature_points['spine1'] = get_data('mus2', 'spine1', h5_file)
mouse2_feature_points['spine2'] = get_data('mus2', 'spine2', h5_file)
mouse2_feature_points['spine3'] = get_data('mus2', 'spine3', h5_file)
mouse2_feature_points['spine4'] = get_data('mus2', 'spine4', h5_file)
mouse2_feature_points['tailbase'] = get_data('mus2', 'tailbase', h5_file)

mouse3_feature_points = {}
mouse3_feature_points['snout'] = get_data('mus3', 'snout', h5_file)
mouse3_feature_points['shoulder'] = get_data('mus3', 'shoulder', h5_file)
mouse3_feature_points['spine1'] = get_data('mus3', 'spine1', h5_file)
mouse3_feature_points['spine2'] = get_data('mus3', 'spine2', h5_file)
mouse3_feature_points['spine3'] = get_data('mus3', 'spine3', h5_file)
mouse3_feature_points['spine4'] = get_data('mus3', 'spine4', h5_file)
mouse3_feature_points['tailbase'] = get_data('mus3', 'tailbase', h5_file)


In [14]:
# male_side_vec = [x, y, width, height]; female_side_vec = [x, y, width, height]
relevant_area = female_side_vec.copy()
# relevant area is female_side_vec + male_side_vec size exactly right next to each other with 50 buffer
relevant_area[2] = female_side_vec[2] + male_side_vec[2] + 50

In [15]:
def torso_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['snout'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['shoulder'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine1'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine2'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine3'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine4'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['tailbase'].loc[[i]]):
    return 1
  else:
    return 0

In [16]:
def snout_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['snout'].loc[[i]]):
    result = 1
  else:
    result = 0
  return result

In [17]:
def shoulder_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['shoulder'].loc[[i]]):
    result = 1
  else:
    result = 0
  return result

# get features code:

In [18]:
def get_i_features(i, first_mouse_feature_points, second_mouse_feature_points):
  if second_mouse_feature_points == 0:
    return np.array([0, -1, 0, -1])

  else:
    if within_area(relevant_area, first_mouse_feature_points['tailbase'].loc[[i]]) and within_area(relevant_area, second_mouse_feature_points['tailbase'].loc[[i]]):
      both_tailbase = 1
      dist1 = euclid_dist(first_mouse_feature_points['tailbase'].loc[[i]], second_mouse_feature_points['tailbase'].loc[[i]])
    else:
      both_tailbase = 0
      dist1 = -1
    
    if within_area(relevant_area, first_mouse_feature_points['snout'].loc[[i]]) and within_area(relevant_area, second_mouse_feature_points['snout'].loc[[i]]):
      both_snout = 1
      dist2 = euclid_dist(first_mouse_feature_points['snout'].loc[[i]], second_mouse_feature_points['snout'].loc[[i]])
    else:
      both_snout = 0
      dist2 = -1
    return np.array([both_tailbase, dist1, both_snout, dist2])

In [19]:
def calculate_i_features(total_frames):
  num_features = 4
  i_features = np.empty([total_frames, num_features])
  
  for i in range(0, total_frames):
    
    # 2 mice detected
    if torso_in_area(relevant_area, 1, i) and torso_in_area(relevant_area, 2, i):
      first_mouse_feature_points = mouse1_feature_points
      second_mouse_feature_points = mouse2_feature_points

    elif torso_in_area(relevant_area, 1, i) and torso_in_area(relevant_area, 3, i):
      first_mouse_feature_points = mouse1_feature_points
      second_mouse_feature_points = mouse3_feature_points

    elif torso_in_area(relevant_area, 2, i) and torso_in_area(relevant_area, 3, i):
      first_mouse_feature_points = mouse2_feature_points
      second_mouse_feature_points = mouse3_feature_points


    # 1 mouse detected
    elif torso_in_area(relevant_area, 1, i):
      first_mouse_feature_points = mouse1_feature_points
      second_mouse_feature_points = 0
      
    elif torso_in_area(relevant_area, 2, i):
      first_mouse_feature_points = mouse2_feature_points
      second_mouse_feature_points = 0
      
    elif torso_in_area(relevant_area, 3, i):
      first_mouse_feature_points = mouse3_feature_points
      second_mouse_feature_points = 0

    # no mouse detected
    else:
      first_mouse_feature_points = 0
      second_mouse_feature_points = 0

    i_features[i, 0:num_features] = get_i_features(i, first_mouse_feature_points, second_mouse_feature_points)
  
  return i_features

In [20]:
def get_input_features(frame_examples, i_features):
  input_features = np.array([i_features[i[0]-29:i[0]+31, :].flatten() for i in frame_examples])

  return np.nan_to_num(input_features, nan=-1)

# calculate i_features

In [21]:
# 3 min
i_features = calculate_i_features(36000)

In [22]:
with open(project_path + "/behaviors/" +video_name+ '_i_features_temporal2.pickle', 'wb') as f:
    pickle.dump(i_features, f)

#frame features

In [41]:
input_frames = [[i] for i in range(30, 35970)]

In [42]:
input_features = get_input_features(input_frames, i_features)

In [43]:
print(input_features.shape)

(35940, 240)


# run classifier analysis

In [44]:
y_pred = classifier.predict(input_features)

In [46]:
print(y_pred.shape)

(35940,)


In [47]:
output_pred = np.hstack((np.array([0]*30), np.array(y_pred), np.array([0]*30)))

In [48]:
print(output_pred.shape)

(36000,)


# extract and save classified mounts

In [55]:
mount_events = pd.DataFrame(index=range(36000), columns=['events_index', 'minutes', 'seconds', 'behavior_events'])

events_n = 0
for i in range(0, 36000):
  if output_pred[i] == 1 and output_pred[i-1] == 0:
    mount_events["events_index"][events_n] = i
    mount_events["minutes"][events_n] = np.floor(i / (30*60))
    mount_events["seconds"][events_n] = round(i / 30) % 60
    mount_events["behavior_events"][events_n] = 1
    events_n = events_n + 1
print(events_n)
mount_events = mount_events.dropna(how='all')

107


In [57]:
print(mount_events[0:20])

   events_index minutes seconds behavior_events
0           390     0.0      13               1
1           634     0.0      21               1
2           646     0.0      22               1
3           666     0.0      22               1
4           674     0.0      22               1
5           684     0.0      23               1
6          1631     0.0      54               1
7          1637     0.0      55               1
8          1713     0.0      57               1
9          1731     0.0      58               1
10         1744     0.0      58               1
11         3541     1.0      58               1
12         3730     2.0       4               1
13         3746     2.0       5               1
14         4355     2.0      25               1
15         4726     2.0      38               1
16         4736     2.0      38               1
17         4746     2.0      38               1
18         4763     2.0      39               1
19         4779     2.0      39         

In [58]:
with open(project_path + '/behaviors/' + video_name + '_classified_mounts.pickle', 'wb') as f:
    pickle.dump(mount_events, f)

In [59]:
import csv
pickle_data = mount_events
with open(project_path + '/temp_classifier_testing/' + video_name +'_classified_mounts.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(pickle_data.columns)
    for i in range(0, pickle_data.shape[0]):
      # write the data
      writer.writerow(pickle_data.values[i,:])

#labelled forest behavior video

In [ ]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14/'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

end_frame = 6000
for video in [project_path + "/videos/PZ71_1.avi"]:
#for video in glob.glob(project_path + "/videos/" + "*.avi"):
  video_name = os.path.splitext(os.path.basename(video))[0]
  
  labeled_video = video_name+'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered_id_labeled.mp4'

  behavior_file = open(output_dir + video_name + '_forestv5_behavior.pickle', 'rb')
  behavior = pickle.load(behavior_file)
  behavior_file.close()

  cap = cv2.VideoCapture(project_path + 'videos/' + labeled_video)

  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  while cap.isOpened():
    ret,frame = cap.read()
    print(frame.shape)
    frame_width = int(frame.shape[1])
    frame_height = int(frame.shape[0])
    break

  output_video = output_dir + video_name + '_forest_behavior_video.mp4'

  out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'MP4V'), 30, (frame_width,frame_height))

  count = 0
  while cap.isOpened():
    ret,frame = cap.read()
    current_behavior = behavior["behavior"][count]
    text_color = (0,0,0)
    if current_behavior == 'male_side':
      text_color = (0, 0, 150)
    elif current_behavior == 'mount':
      text_color = (0, 0, 255)
    elif current_behavior == 'female_side':
      text_color = (50, 0, 0)
    elif current_behavior == 'forest_mount':
      text_color = (0, 255, 0)
    elif current_behavior == 'unclear':
      text_color = (0, 255, 0)
    else:
      print(current_behavior)
      print(count)

    if current_behavior == 'forest_mount' or current_behavior == 'unclear':
      cv2.putText(frame, str(current_behavior), (25, 25), cv2.FONT_HERSHEY_DUPLEX, 1, text_color, 2)
      cv2.putText(frame, str(count), (25, frame_height), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2)
      out.write(frame)

    count = count + 1
    if count > end_frame:
      break
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  cap.release()
  out.release()

(166, 286, 3)
nan
2018
nan
2019
nan
2020
nan
2021
nan
2094
nan
3603
nan
3604
nan
3605
nan
3606
nan
4259
nan
4260
nan
4261
